In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
! pip install konlpy

In [ ]:
# from konlpy.tag import Mecab
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab/
# !bash install_mecab-ko_on_colab190912.sh

In [48]:
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=bb246ce0834a5400b89f74ee87893a7179c227d6993f2b4330917381a08b2d6c
  Stored in directory: /root/.cache/pip/wheels/93/33/b8/bc2256172a415340e34f3c11ef2b0f3f391769000bb74de988
Successfully built hgtk


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from konlpy.tag import Okt
tokenizer=Okt()

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/싼데비스탄/데이터 전처리/real_finish.csv', encoding='UTF-8')

In [7]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205379 entries, 0 to 205378
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         205379 non-null  object
 1   category      205379 non-null  object
 2   sub_category  205379 non-null  object
 3   price         205379 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 6.3+ MB


## 철자 교정

In [39]:
# test

import hgtk
from tqdm import tqdm
from konlpy.tag import Mecab

def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''
    cho_list = []
    jung_list = []
    jong_list = []
    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            
            cho_list.append(cho)
            jung_list.append(jung)
            jong_list.append(jong)

              
            # decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char
                
    return cho_list, jung_list, jong_list


# mecab = Mecab()

# def tokenize_by_jamo(s):
#     return [word_to_jamo(token) for token in mecab.morphs(s)]
     

In [194]:
def standardization(title, category): # 카테고리를 기준으로 표준화
  # 자모로 나눠주기
  cho_list, jung_list, jong_list = word_to_jamo(title) # title
  cat_cho_list, cat_jung_list, cat_jong_list = word_to_jamo(category) # category

  # 만약 카테고리가 메뉴명 보다 더 길다면 그냥 덮어주기
  if len(cat_cho_list) > len(cho_list):
    cho_list = cat_cho_list
    jung_list = cat_jung_list
    jong_list = cat_jong_list
    return cho_list, jung_list, jong_list

  flag = 0
  for i in tqdm(range(len(cho_list) - len(cat_cho_list) + 1)): # 한칸씩 지나가면서 검사
    old = cho_list[i:len(cat_cho_list)+i] # 카테고리 길이로 슬라이싱
    matching = np.char.equal(old, cat_cho_list) # 일치하는지 bool 값으로 검사
    kyo = matching.sum() # True 값(일치하는 값) 총 개수
    
    # 메뉴명과 카테고리의 길이의 차이가 없거나, 적은 경우
    # 카테고리로 메뉴명을 전부 덮어버리는 것을 방지
    if len(cho_list) - len(cat_cho_list) < 2:
      ## 생각중...
      ## 초성 뿐만 아니라 중성,종성 비교 필요할 수도..
      pass  
    
    if kyo >= len(cat_cho_list)*0.5: # 초성이 카테고리와 50%이상 일치하면 초중종성 전부대체
      cho_list[i:len(cat_cho_list)+i] = cat_cho_list
      jung_list[i:len(cat_cho_list)+i] = cat_jung_list
      jong_list[i:len(cat_cho_list)+i] = cat_jong_list
      break
    else:
      flag += 1

    # 아예 중복 없이 그냥 쭉 지나갔다면 뒤에다가 카테고리 그대로 추가
    if flag == len(cho_list) - len(cat_cho_list) + 1:
      cho_list += cat_cho_list
      jung_list += cat_jung_list
      jong_list += cat_jong_list

  return cho_list, jung_list, jong_list

In [142]:
## 뒤에 붙은 메뉴명 표준화
cho_list, jung_list, jong_list = standardization('맛있는 마 뭐라카노', '아메리카노')

 75%|███████▌  | 3/4 [00:00<00:00, 10681.59it/s]


In [143]:
cho_list

['ㅁ', 'ㅇ', 'ㄴ', 'ㅇ', 'ㅁ', 'ㄹ', 'ㅋ', 'ㄴ']

In [144]:
jung_list

['ㅏ', 'ㅣ', 'ㅡ', 'ㅏ', 'ㅔ', 'ㅣ', 'ㅏ', 'ㅗ']

In [145]:
jong_list

['ㅅ', 'ㅆ', 'ㄴ', '-', '-', '-', '-', '-']

In [165]:
## 앞에 붙은 메뉴명 표준화
cho_list, jung_list, jong_list = standardization('코히라퉤 먹자', '커피라떼')

  0%|          | 0/3 [00:00<?, ?it/s]


In [166]:
cho_list

['ㅋ', 'ㅍ', 'ㄹ', 'ㄸ', 'ㅁ', 'ㅈ']

In [167]:
jung_list

['ㅓ', 'ㅣ', 'ㅏ', 'ㅔ', 'ㅓ', 'ㅏ']

In [168]:
jong_list

['-', '-', '-', '-', 'ㄱ', '-']

In [161]:
## 가운데에 있는 메뉴명 표준화
cho_list, jung_list, jong_list = standardization('인기메뉴 코구마시발듸 세트', '고구마스무디')

 57%|█████▋    | 4/7 [00:00<00:00, 8008.22it/s]


In [162]:
cho_list

['ㅇ', 'ㄱ', 'ㅁ', 'ㄴ', 'ㄱ', 'ㄱ', 'ㅁ', 'ㅅ', 'ㅁ', 'ㄷ', 'ㅅ', 'ㅌ']

In [163]:
jung_list

['ㅣ', 'ㅣ', 'ㅔ', 'ㅠ', 'ㅗ', 'ㅜ', 'ㅏ', 'ㅡ', 'ㅜ', 'ㅣ', 'ㅔ', 'ㅡ']

In [164]:
jong_list

['ㄴ', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']

In [150]:
## 아예 없는 메뉴명엔 카테고리 추가
cho_list, jung_list, jong_list = standardization('예가체프', '블랜딩')

100%|██████████| 2/2 [00:00<00:00, 6517.95it/s]


In [151]:
cho_list

['ㅇ', 'ㄱ', 'ㅊ', 'ㅍ', 'ㅂ', 'ㄹ', 'ㄷ']

In [152]:
jung_list

['ㅖ', 'ㅏ', 'ㅔ', 'ㅡ', 'ㅡ', 'ㅐ', 'ㅣ']

In [154]:
jong_list

['-', '-', '-', '-', 'ㄹ', 'ㄴ', 'ㅇ']

In [199]:
## 이 밑으로는 예외처리 추가/수정 필요

In [195]:
## 메뉴명 보다 카테고리가 더 길다면 그냥 대체
cho_list, jung_list, jong_list = standardization('국시', '전통국수')

In [196]:
cho_list

['ㅈ', 'ㅌ', 'ㄱ', 'ㅅ']

In [197]:
jung_list

['ㅓ', 'ㅗ', 'ㅜ', 'ㅜ']

In [198]:
jong_list

['ㄴ', 'ㅇ', 'ㄱ', '-']

In [ ]:
## 메뉴명과 카테고리의 길이의 차이가 없거나, 적은 경우
cho_list, jung_list, jong_list = standardization('멸치쌀국시', '전통국수') # '멸치쌀국수'로 표준화 되어야함

## TF-IDF

In [34]:
coffee = df[df['sub_category']=='커피']

In [31]:
coffee

,title,category,sub_category,price
39034,동서식품 맥심 모카골드 마일드,생수/음료,커피,10800
39035,베트남 수출용 블랙커피,생수/음료,커피,12490
39036,동서식품 맥심 모카 골드 마일드,생수/음료,커피,24800
39037,동서식품 맥스웰하우스 커피믹스 마일드 자판기용,생수/음료,커피,4510
39038,동서식품 맥심 디카페인 커피믹스,생수/음료,커피,15420
...,...,...,...,...
152491,다크브릭스 드립백 커피,커피/차류,커피,7000
152492,폴바셋 바리스타 파우치 시그니처 블렌드,커피/차류,커피,11900
152493,신선한 원두 고급블랜딩 드립백 티백커피,커피/차류,커피,12000
152494,칸타타 드립 단품,커피/차류,커피,91200


In [35]:
tit = coffee.title

In [36]:
title = []
for t in tqdm(tit):
  title.append(tokenizer.nouns(t))

100%|██████████| 8921/8921 [00:14<00:00, 635.38it/s]


In [37]:
## 토큰들 문자열로 결합
def tokenToString(token_list):
    res = []
    for token in tqdm(token_list):
      s = ' '.join(token)
      res.append(s)
    return res
titles = tokenToString(title)

100%|██████████| 8921/8921 [00:00<00:00, 1272830.08it/s]


In [ ]:
titles

In [39]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(titles)
tf = pd.DataFrame(dtm.toarray(), columns = vectorizer.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
tf

,가나,가능,가득,가든,가루,가리발디,가미,가방,가배,가성,...,흑임자,흥국,희망,희창,히든,히비스커스,히스토리,히즈빈스,히트,힐링
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
bindo = tf.astype(bool).sum(axis = 0)

In [42]:
## 문서 개수
D = len(tf)

## Inverse Document Frequency
idf = np.log((D+1) / (bindo+1)) + 1

In [43]:
## TF-IDF
tfidf = tf * idf                      
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)

In [44]:
tfidf

,가나,가능,가득,가든,가루,가리발디,가미,가방,가배,가성,...,흑임자,흥국,희망,희창,히든,히비스커스,히스토리,히즈빈스,히트,힐링
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
dd = pd.DataFrame(tfidf.max().sort_values(ascending=False))

In [46]:
dd

,0
리그,1.000000
커피,1.000000
나무,1.000000
오설록,1.000000
쟈뎅,1.000000
...,...
액세서리,0.286321
기입,0.284733
그룹,0.284733
졸업,0.284733
